In [256]:
import numpy as np
from copy import copy

In [257]:
def simulate_episode(init_prob_matrix, n_steps_max, initial_active_node):
    prob_matrix = init_prob_matrix.copy()
    history = np.array([initial_active_node])
    active_nodes = np.array([initial_active_node]) # array di active nodes esempio: se active_nodes sono 2 e 3 allora sarà [[0,1,0,0,0], [0,0,1,0,0]]
    newly_active_nodes = active_nodes
    t = 0
    print(active_nodes, end='\n\n')
    while (t < n_steps_max and np.sum(newly_active_nodes) > 0):
        active_node = active_nodes[0].copy() # Prendi il primo active node in coda
        active_nodes = active_nodes[1:]
        print(prob_matrix, end='\n')
        print(active_node)

        p = (prob_matrix.T * active_node).T
        activated_edges = p > np.random.rand(p.shape[0], p.shape[1])  # AGGIUNGERE I RESERVATION PRICE E LAMBDA
        #prob_matrix = prob_matrix * ((p != 0) == activated_edges)
        idx_active = np.argwhere(active_node)
        prob_matrix[:, idx_active] = 0
        newly_active_nodes = (np.sum(activated_edges, axis=0) > 0) * (1 - active_node)

        for idx, val in enumerate(newly_active_nodes):
            if val == 1:
                a = np.zeros(5)
                a[idx] = 1
                active_nodes = np.concatenate((active_nodes, [a]), axis=0)
        #print(active_nodes, end='\n\n')
        active_nodes = np.unique(active_nodes, axis=0)
        history = np.concatenate((history, [active_node]), axis=0)
        t += 1
    print(prob_matrix)
    return history

In [258]:
def estimate_probabilities(dataset, node_index, n_nodes):
    estimated_prob = np.ones(n_nodes) * 1.0 / (n_nodes - 1)
    credits = np.zeros(n_nodes)
    occur_v_active = np.zeros(n_nodes)
    n_episodes = len(dataset)
    for episode in dataset:
        idx_w_active = np.argwhere(episode[:, node_index] == 1).reshape(-1)
        if len(idx_w_active) > 0 and idx_w_active > 0:
            active_nodes_in_prev_step = episode[idx_w_active - 1, :].reshape(-1)
            credits += active_nodes_in_prev_step / np.sum(active_nodes_in_prev_step)
        for v in range(0, n_nodes):
            if (v != node_index):
                idx_v_active = np.argwhere(episode[:, v] == 1).reshape(-1)
                if len(idx_v_active) > 0 and (idx_v_active < idx_w_active or len(idx_w_active) == 0):
                    occur_v_active[v] += 1
    estimated_prob = credits / occur_v_active
    estimated_prob = np.nan_to_num(estimated_prob)
    return estimated_prob

In [259]:
n_nodes = 5
n_episodes = 1000
prob_matrix = np.random.uniform(0.0, 0.9, (n_nodes, n_nodes))

nodes = np.array(range(n_nodes+1))
alphas = np.random.dirichlet(np.ones(6), size=1).reshape(-1)
print(alphas)

[0.03301081 0.0096459  0.40373719 0.17062718 0.30823982 0.07473909]


In [260]:
dataset = []
node_index = np.array([4])

for e in range(0, 1):
    initial_node = np.random.choice(nodes, 1, p=alphas)
    initial_active_node = np.zeros(n_nodes+1)
    initial_active_node[initial_node] = 1
    initial_active_node = initial_active_node[1:]
    dataset.append(simulate_episode(init_prob_matrix=prob_matrix, n_steps_max=10, initial_active_node=initial_active_node))
    print(dataset)
    
for n in node_index:
    estimated_prob = estimate_probabilities(dataset=dataset, node_index=n, n_nodes=n_nodes)
    print(f"{n}:{estimated_prob}")

print("True P Matrix: ")
print(prob_matrix.T)


[[0. 0. 0. 1. 0.]]

[[0.83341472 0.27581062 0.29891048 0.74454813 0.63350268]
 [0.25987684 0.2498823  0.18951515 0.37318673 0.05531461]
 [0.7144913  0.76763421 0.78233106 0.1758947  0.10697677]
 [0.89295093 0.35593285 0.52793943 0.50562442 0.82641027]
 [0.61940431 0.40497553 0.23731897 0.64269717 0.37069272]]
[0. 0. 0. 1. 0.]
[[0.83341472 0.27581062 0.29891048 0.         0.63350268]
 [0.25987684 0.2498823  0.18951515 0.         0.05531461]
 [0.7144913  0.76763421 0.78233106 0.         0.10697677]
 [0.89295093 0.35593285 0.52793943 0.         0.82641027]
 [0.61940431 0.40497553 0.23731897 0.         0.37069272]]
[1. 0. 0. 0. 0.]
[[0.         0.27581062 0.29891048 0.         0.63350268]
 [0.         0.2498823  0.18951515 0.         0.05531461]
 [0.         0.76763421 0.78233106 0.         0.10697677]
 [0.         0.35593285 0.52793943 0.         0.82641027]
 [0.         0.40497553 0.23731897 0.         0.37069272]]
[0. 1. 0. 0. 0.]
[[0.         0.         0.29891048 0.         0.63350268

c:\Python37\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  


ValueError: operands could not be broadcast together with shapes (2,) (0,) 